In [22]:
from dotenv import load_dotenv
from pinecone import Pinecone
import os
import google.generativeai as genai

load_dotenv()

api_key = os.getenv("PINE_CONE_KEY")
pc = Pinecone(api_key = api_key)
index = pc.Index("rag")

api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

In [23]:
user_query = "What are the key things to consider when working on F1 Stem opt " 

query_vector = genai.embed_content(
    model="models/embedding-001",   # same model as you used before
    content=user_query
)["embedding"]

In [24]:
print(query_vector)

[-0.0060856626, -0.09055217, -0.041418403, -0.068905726, 0.056084067, 0.03704344, -0.020833861, -0.017179484, -0.027870877, 0.012690554, 0.03226476, -0.00068110815, -0.023542423, 0.021380402, 0.035143357, -0.038838167, 0.02161672, -0.014981312, -0.015205486, -0.033725128, -0.028842917, 0.019137852, 0.044653393, -0.0026734313, -0.007508047, 0.000692931, 0.0038232473, -0.09664589, 0.0097160945, 0.06833676, -0.08004796, 0.040081713, -0.092318065, 0.0007946156, 0.016716179, -0.08051668, -0.0045873243, 0.028710231, -0.04530112, -0.0064452933, -0.024486277, -0.0071459967, -0.028172458, -0.04165066, -0.014726559, -0.042759754, 0.011725587, 0.058349267, -0.047388688, -0.033918142, 0.010891707, -0.021471864, 0.05101163, -0.013043907, -0.0027179401, 0.0018701392, 0.042366456, -0.058688313, 0.043816023, 0.028003061, 0.002098559, 0.012352637, -0.017748179, 0.035855755, 0.02872987, -0.04905246, -0.013439708, 0.004669709, 0.03734715, -0.03625553, -0.020502482, -0.006826968, 0.023887051, -0.008292474

In [25]:
user_query = "What are the key things to consider when working on F1 Stem opt " 

query_vector = genai.embed_content(
    model="models/embedding-001",   # same model as you used before
    content=user_query
)["embedding"]
results = index.query(
    vector=query_vector,
    top_k = 10,                # number of most relevant chunks to fetch
    include_metadata=True,  # so you also get back your original text
)

In [26]:
print(results)

{'matches': [{'id': 'doc_524_chunk1',
              'metadata': {'chunk': 'Key insights include:',
                           'original_text_length': 18052.0,
                           'source': 'USCIS website',
                           'timestamp': '2025-10-22T16:54:32.344580',
                           'url': '/www.uscis.gov/sites/default/files/document/data/all_forms_performancedata_fy2015_qtr4.html'},
              'score': 0.633652329,
              'values': []},
             {'id': 'doc_447_chunk1',
              'metadata': {'chunk': '**Key points include:**',
                           'original_text_length': 41725.0,
                           'source': 'USCIS website',
                           'timestamp': '2025-10-22T16:52:42.073898',
                           'url': '/www.uscis.gov/policy-manual/volume-8-part-g-chapter-7.html'},
              'score': 0.625945687,
              'values': []},
             {'id': 'doc_822_chunk1',
              'metadata': {'chunk': 

In [29]:
text = ""
for match in results["matches"]:
    score = match["score"]             # similarity score
    text += match["metadata"]["chunk"]   # your stored chunk
    chunk_id = match["id"]   # original doc ID
    
    print(f"Score: {score:.4f} | Doc: {chunk_id}")
    print(f"Chunk: {text[:200]}...\n")

Score: 0.6337 | Doc: doc_524_chunk1
Chunk: Key insights include:...

Score: 0.6259 | Doc: doc_447_chunk1
Chunk: Key insights include:**Key points include:**...

Score: 0.6217 | Doc: doc_822_chunk1
Chunk: Key insights include:**Key points include:***   **F-1 OPT (선택적 실습 교육):** F-1 학생에게 STEM 관련 분야에서 최대 36개월(초기 OPT 12개월 + STEM OPT 24개월)의 취업 허가를 제공합니다. STEM OPT 연장에는 고용 제안이 필요합니다.
*   **H-1B (전문 직종):** 관련 ...

Score: 0.6198 | Doc: doc_821_chunk0
Chunk: Key insights include:**Key points include:***   **F-1 OPT (선택적 실습 교육):** F-1 학생에게 STEM 관련 분야에서 최대 36개월(초기 OPT 12개월 + STEM OPT 24개월)의 취업 허가를 제공합니다. STEM OPT 연장에는 고용 제안이 필요합니다.
*   **H-1B (전문 직종):** 관련 ...

Score: 0.6158 | Doc: doc_824_chunk0
Chunk: Key insights include:**Key points include:***   **F-1 OPT (선택적 실습 교육):** F-1 학생에게 STEM 관련 분야에서 최대 36개월(초기 OPT 12개월 + STEM OPT 24개월)의 취업 허가를 제공합니다. STEM OPT 연장에는 고용 제안이 필요합니다.
*   **H-1B (전문 직종):** 관련 ...

Score: 0.6071 | Doc: doc_821_chunk5
Chunk: Key insights include:**Key points include:***   **F-

In [7]:
print(text)

In [19]:
print(len(text))

1767


In [30]:
system_prompt = f""" You are a helpful agent for international students use the following information to provide the answer better use the context to answer the question.
context: {text}
"""

In [ ]:
system_prompt = f""" You are a helpful agent for international students use the following information to provide the answer better use the context to answer the question.
context: {text}
"""
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=system_prompt,
)
response = model.generate_content(user_query)
print(response)

In [32]:
response = model.generate_content(user_query)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "When considering F-1 STEM OPT, here are the key things from the provided context:\n\n*   **Eligibility:**\n    *   You must be an F-1 student currently in a valid OPT period.\n    *   You must have earned a STEM degree from an accredited and SEVP-certified institution.\n    *   Previously obtained STEM degrees or future higher-level STEM degrees can also be used for extensions.\n*   **Duration:**\n    *   It provides up to 12 months of initial OPT.\n    *   There's a potential 24-month STEM OPT extension, bringing the total to a maximum of 36 months for STEM-related fields.\n*   **Employment Requirements:**\n    *   You need to secure employment with an employer that meets specific requirements.\n    *   An employment offer is required for the STEM OPT exten

In [33]:
response.text

"When considering F-1 STEM OPT, here are the key things from the provided context:\n\n*   **Eligibility:**\n    *   You must be an F-1 student currently in a valid OPT period.\n    *   You must have earned a STEM degree from an accredited and SEVP-certified institution.\n    *   Previously obtained STEM degrees or future higher-level STEM degrees can also be used for extensions.\n*   **Duration:**\n    *   It provides up to 12 months of initial OPT.\n    *   There's a potential 24-month STEM OPT extension, bringing the total to a maximum of 36 months for STEM-related fields.\n*   **Employment Requirements:**\n    *   You need to secure employment with an employer that meets specific requirements.\n    *   An employment offer is required for the STEM OPT extension.\n    *   There must be a genuine employer-employee relationship, which may be subject to scrutiny.\n    *   Employers must complete Form I-983 (Training Plan for STEM OPT Students).\n    *   DHS may verify employer responsibi